# Fine-Tuning DistilBERT with LoRA for IMDB Sentiment Classification

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install transformers datasets peft accelerate evaluate torch

In [ ]:
import os, random, numpy as np, torch
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from peft import LoraConfig, TaskType, get_peft_model

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)

In [ ]:
dataset = load_dataset('imdb')
SUB_TRAIN, SUB_TEST = 8000, 2000
dataset['train'] = dataset['train'].shuffle(seed=SEED).select(range(SUB_TRAIN))
dataset['test'] = dataset['test'].shuffle(seed=SEED).select(range(SUB_TEST))

split = dataset['train'].train_test_split(test_size=0.1, seed=SEED)
train, val, test = split['train'], split['test'], dataset['test']

In [ ]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=256)

train_tok = train.map(tokenize, batched=True, remove_columns=['text'])
val_tok   = val.map(tokenize, batched=True, remove_columns=['text'])
test_tok  = test.map(tokenize, batched=True, remove_columns=['text'])

for ds in [train_tok, val_tok, test_tok]:
    ds.set_format(type='torch', columns=['input_ids','attention_mask','label'])

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

lora = LoraConfig(task_type=TaskType.SEQ_CLS, r=8, lora_alpha=16, lora_dropout=0.1,
                  target_modules=['q_lin','v_lin'])
model = get_peft_model(model, lora).to(device)

def print_trainable(model):
    t, a = 0, 0
    for p in model.parameters():
        a += p.numel()
        if p.requires_grad: t += p.numel()
    print(f'Trainable: {t} / {a} ({100*t/a:.2f}%)')

print_trainable(model)

In [ ]:
acc = evaluate.load('accuracy')
f1 = evaluate.load('f1')
def metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        'accuracy': acc.compute(predictions=preds, references=labels)['accuracy'],
        'f1': f1.compute(predictions=preds, references=labels)['f1'],
    }

In [ ]:
args = TrainingArguments(
    output_dir='lora-distilbert-imdb',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    learning_rate=5e-4,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy'
)

trainer = Trainer(model=model, args=args, train_dataset=train_tok,
                  eval_dataset=val_tok, tokenizer=tokenizer,
                  data_collator=data_collator, compute_metrics=metrics)

In [ ]:
trainer.train()

In [ ]:
print('Validation:', trainer.evaluate(eval_dataset=val_tok))
print('Test:', trainer.evaluate(eval_dataset=test_tok))

In [ ]:
def classify(text):
    model.eval()
    inputs = tokenizer(text, return_tensors='pt', truncation=True,
                       padding=True, max_length=256).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]
    label = 'pos' if np.argmax(probs)==1 else 'neg'
    return {'text': text, 'label': label, 'confidence': float(max(probs))}

print(classify('This movie was fantastic!'))
print(classify('The movie had moments but was boring overall.'))